In [1]:
pip install tensorflow keras

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Load and preprocess your data (replace with actual dataset)
data = pd.read_csv(r'C:\Users\TUF GAMING\Documents\main_dataset.csv')

# Feature engineering (like converting columns, cleaning, creating dummy variables, etc.)
# Example of converting 'Industry' to dummy variables
industry_dummies = pd.get_dummies(data['INDUSTRY'], drop_first=True)

# Remove '$', 'Billion', and 'Million', and convert to numeric values
data['CURRENT VALUATION'] = data['CURRENT VALUATION'].replace({'\$': '', ' Billion': 'e9', ' Million': 'e6'}, regex=True)

# Convert to float
data['CURRENT VALUATION'] = pd.to_numeric(data['CURRENT VALUATION'], errors='coerce')

# Combine with other features
X = pd.concat([data[['FOUNDING YEAR', 'UNICORN ENTRY YEAR', 'ACQUISITIONS']], industry_dummies], axis=1)
y = (data['CURRENT VALUATION'] > 1).astype(int)  # Define success as valuation > $1 billion

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Build the ANN
model_ann = Sequential()

# Input and hidden layers
model_ann.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model_ann.add(Dense(units=32, activation='relu'))

# Output layer for binary classification
model_ann.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model_ann.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model_ann.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model_ann.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/50


C:\Users\TUF GAMING\anaconda3\envs\py3117_20240115\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.7471 - loss: 0.6219 - val_accuracy: 1.0000 - val_loss: 0.6894
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.9770 - loss: 0.6890 - val_accuracy: 1.0000 - val_loss: 0.6855
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9874 - loss: 0.6853 - val_accuracy: 1.0000 - val_loss: 0.6816
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9770 - loss: 0.6816 - val_accuracy: 1.0000 - val_loss: 0.6777
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9770 - loss: 0.6779 - val_accuracy: 1.0000 - val_loss: 0.6737
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9874 - loss: 0.6737 - val_accuracy: 1.0000 - val_loss: 0.6697
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9874 - loss: 0.6699 - val_accuracy: 1.0000 - val_loss: 0.6657
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.9874 - loss: 0.6660 - val_accuracy: 1.0000 - val_loss: 0.6617
Epoch 9/50

In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam

# Assuming the model is already trained and scaler is fitted

# Example Input for a startup
company_name = "Example Startup"
sector = "E-commerce"
founding_year = 2022
entry_year = 2025
valuation = 500e6  # $500 Million converted to numerical

# Build the ANN model using Input layer to specify shape
model_ann = Sequential()

# Input layer (define input shape)
model_ann.add(Input(shape=(X_train.shape[1],)))

# Hidden layers
model_ann.add(Dense(units=64, activation='relu'))
model_ann.add(Dense(units=32, activation='relu'))

# Output layer for binary classification
model_ann.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model_ann.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Example new input for prediction
new_data = pd.DataFrame({
    'FOUNDING YEAR': [founding_year],
    'UNICORN ENTRY YEAR': [entry_year],
    'ACQUISITIONS': [0],  # Set to 0 or provide actual number if available
    'CURRENT VALUATION': [valuation],
})

# Add industry dummy variables as in training set
industry_dummies = pd.get_dummies(pd.Series([sector]), prefix='INDUSTRY', drop_first=True)
new_data = pd.concat([new_data, industry_dummies], axis=1)

# Ensure the new data has the same structure as the training data
for col in X.columns:
    if col not in new_data.columns:
        new_data[col] = 0  # Add missing columns

# Ensure columns are in the same order as training data
new_data = new_data[X.columns]

# Scale the new input using the same scaler as before
new_data_scaled = scaler.transform(new_data)

# Predict success or failure for the new startup
prediction_proba = model_ann.predict(new_data_scaled)

# Print the prediction probabilities (success rate and failure rate)
success_rate = prediction_proba[0][0] * 100
failure_rate = (1 - prediction_proba[0][0]) * 100

print(f"Startup '{company_name}' in the sector '{sector}' has a:")
print(f"- Success rate: {success_rate:.2f}%")
print(f"- Failure rate: {failure_rate:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Startup 'Example Startup' in the sector 'E-commerce' has a:
- Success rate: 62.82%
- Failure rate: 37.18%
